### Analyzing Word Frequencies in Text
Objectives:
- Calculating basic statistics of word frequencies
- Visualize the frequency distribution of common words
- Compare observed word frequencies with expected distributions
- Draw insights about word usage pattern in the dataset

In [2]:
# Load and preprocess text data
import re # allows you to use regular expressions (regex) in Python.
import nltk
nltk.download('punkt')
import spacy
import gensim
import string
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation


[nltk_data] Downloading package punkt to /home/wanyua/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Load SpaCy's English model
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('average_perceptron_tagger')
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /home/wanyua/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/wanyua/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wanyua/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
